## Setup

In [27]:
#pip install --upgrade git+https://github.com/keras-team/keras-cv -q

In [1]:
from tqdm.auto import tqdm
import tensorflow as tf
import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

/home/maog77/.virtualenvs/TF_GPU/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-29 17:33:02.141666: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-29 17:33:02.453849: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-29 17:33:02.453894: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-29 17:33:0

Using TensorFlow backend


In [2]:
class_ids = [
    #"Chair","Sofa","Table"
    #"TV", "bed", "chair", "clock", "console", "consoleeeeee", "door", "fan", "light", "sofa", "switchboard", "table"
    #"emptychair", "fullchair"
    "Cane", "Gatto"
]
class_mapping = dict(zip(range(len(class_ids)), class_ids))

In [3]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset(
    "yolo_v8_m_backbone_coco",  # We will use yolov8 small backbone with coco weights
    load_weights=True
)
backbone.trainable = False

2024-02-29 17:33:16.196669: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 17:33:16.363383: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 17:33:16.363426: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 17:33:16.366906: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-29 17:33:16.366953: I external/local_xla/xla/stream_executor

In [4]:
yolo = keras_cv.models.YOLOV8Detector(
    num_classes=len(class_mapping),
    bounding_box_format="xyxy",
    backbone=backbone,
    #fpn_depth=1
)

In [5]:
DATASET_NAME = "Animali"
BACKBONE_SIZE = "m"
BACKBONE_NAME ="yolo_v8_" + BACKBONE_SIZE + "_backbone_coco"
BACKBONE_TRAINABLE = False

MODEL_NAME = "model-{size}{freezed}-{dataset}".format(size=BACKBONE_SIZE, freezed= "-bbfreez" if BACKBONE_TRAINABLE == False else "", dataset=DATASET_NAME)

path_dataset = "../../Yolo8_Dataset"
   
path_model = path_dataset + F"/{DATASET_NAME}/models/" + MODEL_NAME + ".h5"


yolo.load_weights(path_model)
#yolo = keras_cv.models.load("model.h5")

In [6]:
path_immagini_source = path_dataset + F"/{DATASET_NAME}/source/"

In [7]:
yolo.prediction_decoder = keras_cv.layers.NonMaxSuppression(
     bounding_box_format="xyxy",
     from_logits=False,
     iou_threshold=0.10,
     confidence_threshold=0.01,
     max_detections=50
 )

In [10]:
def visualize_generic(model, image_name, bounding_box_format):
    #images = load_image(os.path.join(path_images, image_name))
    image = tf.io.read_file(image_name)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [640,640])
    images = tf.cast(image, tf.float32)
    images = tf.expand_dims(images, axis=0)

    y_pred = model.predict(images)
    y_pred = bounding_box.to_ragged(y_pred)
    visualization.plot_bounding_box_gallery(
        images,
        value_range=(0, 255),
        bounding_box_format=bounding_box_format,
        #y_true={},
        y_pred=y_pred,
        scale=4,
        rows=1,
        cols=1,
        show=True,
        font_scale=0.7,
        class_mapping=class_mapping,
    )


# visualize_generic(yolo, image_name="../dataset1/test/00012_jpg.rf.f54b7ef0968cf6b0f990928d169e1a83.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset1/test/00041_jpg.rf.1b685611d13e60bfd19b1ad008eb81c9.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset1/test/00059_jpg.rf.f69bbdb4d09580675715e9a203470631.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset3/test/Chair-269-_jpg.rf.3a81b0889599ce42bc735ff199f7e536.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset3/test/Sofa-359-_jpg.rf.2995061f9eaae4bca7913b736e3e49a1.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../dataset3/test/Chairs1-16-_jpg.rf.a4506ce899294f71c7fd534a42534144.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_1.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_2.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_3.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_4.jpg", bounding_box_format="xyxy")
# visualize_generic(yolo, image_name="../test_images/testimage_5.jpg", bounding_box_format="xyxy")

visualize_generic(yolo, image_name=path_immagini_source + "/cane-e-gatto-insieme.jpg", bounding_box_format="xyxy")
visualize_generic(yolo, image_name=path_immagini_source + "/183005673-110464f6-fd3c-42b8-9c38-553f0e46a50d.jpg", bounding_box_format="xyxy")
visualize_generic(yolo, image_name=path_immagini_source + "/cani-gatti-litigano-m.jpg", bounding_box_format="xyxy")
visualize_generic(yolo, image_name=path_immagini_source + "/dog-2606759_640.jpg", bounding_box_format="xyxy")

visualize_generic(yolo, image_name=path_immagini_source + "/billy001.jpg", bounding_box_format="xyxy")
visualize_generic(yolo, image_name=path_immagini_source + "/billy002.jpg", bounding_box_format="xyxy")


#img_0120_jpg.rf.36da0f1b2f59ec13cb70e1b82defd9a0.jpg

InvalidArgumentError: Graph execution error:

Detected at node yolov8_detector/tf.concat_5/concat defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/maog77/.virtualenvs/TF_GPU/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/maog77/.virtualenvs/TF_GPU/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/maog77/.virtualenvs/TF_GPU/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/maog77/.virtualenvs/TF_GPU/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/home/maog77/.